In [1219]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [683]:
class dual_svm:
    def __init__(self):
        self.epsilon = 1

    def check_constraint(self, y, alpha):
        length, _ = y.shape
        summation = 0
        
        # Generating alphas from 2 to m
        for i in range(1, length):
            summation += alpha[i] * y[i] * y[0]
            
        # Checking if the constraint is met or not    
        if -summation >= 0:
            return -summation
        else:
            return False
    
    def differentiation(self, x, y, alphas, i):
        length = alphas.shape[0]
        
        # Getting the summations ready
        first_summation, second_summation, third_summation, fourth_summation = 0, 0, 0, 0
        for j in range(1, length):
            first_summation += np.dot(alphas[j], (np.dot(y[j], y[0])))
            second_summation += np.dot(alphas[j], (np.dot(y[j], y[0])))
            third_summation += np.dot(alphas[j], np.dot(y[j], (1 + np.dot(x[j], x[0]))**2))
            if j != i:
                fourth_summation += np.dot(alphas[j], (np.dot(y[j], y[i])))
                
        # Third summation is a vector
        current_y_terms = np.dot(y[i], y[0])
        current_x_terms = (1 + np.dot(x[i], x[0]))**2
        
        # This is the 'a' part that would be returned
        final_term_svm = -current_y_terms + 1 - (first_summation * current_y_terms * 1 + (1 + np.dot(x[1], x[1]))**2) + y[0] \
        * ((first_summation * y[i] * current_x_terms) + (third_summation * current_y_terms)) \
        -((alphas[i] * (1 + np.dot(x[i], x[i]))**2) + (fourth_summation * (1 + np.dot(x[i], x[j]))**2))
        
        
        # This is the 'b' part to be returned
        final_term_barrier = ((1/alphas[i]) - ((y[i] * y[0]) / first_summation))
        
        return final_term_svm, final_term_barrier
        
    
    def update_alphas(self, x, y ,alphas, time):
        
        alpha_lengths = alphas.shape[0]
        summation = 0
        
        for i in range(1, alpha_lengths):
            # Finding differentiation of SVM part (a) and the log barrier part (b)
            a, b = self.differentiation(x, y, alphas, i)
            
            # Epsilon/time to decrease the power of barrier
            step_size = 0.001 * (a[0] + ((self.epsilon / time**2) * b[0]))
#             if abs(step_size) < 0.001:
#                 continue
#             else:
#                 alphas[i] -= step_size
            alphas[i] -= step_size
                
        for i in range(1, alpha_lengths):
            summation += alphas[i] * y[i] * y[0]
        alphas[0] = -summation
        
        return
            
        
    def barrier_svm(self, x, y):
        m, k = x.shape
        total_iterations = 100
        
        # Generating alphas satisfying the constraint
        while True:
            alpha_array = np.random.random_sample((m,))
            alpha1 = self.check_constraint(y, alpha_array)
            
            # If constraint is met, we change alpha_1 and break. Otherwise we continue
            if alpha1:
                alpha_array[0] = alpha1
                break
            
        # Got the initial feasible choice for alphas at t=0
        # Now computing the unconstrained maximizer using SVM's changes equation and barrier method.
        # self.barrier_equation(y, alpha_array)
        
        print(alpha_array)
        
        for time in range(1, total_iterations):
#             print(time)
            # Using time to decrease the power of barrier (epsilon)
            
            self.update_alphas(x, y, alpha_array, time)
        
        print(alpha_array)
        

In [809]:
# Experimenting on this
class dual_svm:
    def __init__(self):
        self.epsilon = 1

    def check_constraint(self, y, alpha):
        length, _ = y.shape
        summation = 0
        
        # Generating alphas from 2 to m
        for i in range(1, length):
            summation += alpha[i] * y[i] * y[0]
            
        # Checking if the constraint is met or not    
        if -summation >= 0:
            return -summation
        else:
            return False
        
    def kernel_function(self, x1, x2):
        return (1 + np.dot(x1, x2))**2
    
    # ----------------------------------------------------------------------------------------------------------
    
#     def differentiation(self, x, y, alphas, j):
#         length = alphas.shape[0]
        
#         first_summation, second_summation, third = 0, 0, 0
#         for i in range(1, length):
#             first_summation += (alphas[i] * y[i][0] * y[j][0])
#             second_summation += (alphas[i] * y[i][0])
#             third += (alphas[i] * y[i][0] * y[j][0] * self.kernel_function(x[i], x[j]))
        
#         final_term_svm = -(y[0][0] * y[j][0]) + 1 - (first_summation * self.kernel_function(x[i], x[j])) + (first_summation * self.kernel_function(x[j], x[0])) + (first_summation * self.kernel_function(x[i], x[0])) + third
        
#         final_term_barrier = (1/alphas[j]) + (y[j][0]/second_summation)
        
#         print("Theta is ",-final_term_barrier)
#         return final_term_svm, -final_term_barrier
        
    
    # -------------------------------------------------------------------------------------------------------
    def update_alphas(self, x, y ,alpha_values, time):
        
        length = alpha_values.shape[0]
        summation = 0
        temp_alphas = []
        for i in range(1, length):
            final_svm, final_barrier = 0, 0
            first_summation, second_summation, third_summation = 0, 0, 0
            for j in range(1, length):
                first_summation += alpha_values[j] * y[j][0]
                second_summation += alpha_values[j] * y[j][0] * self.kernel_function(x[j], x[0])
                third_summation += alpha_values[j] * y[j][0] * self.kernel_function(x[j], x[i]) * y[i][0]
            
            term1 = y[i][0] * y[0][0]
            term2 = 1
            term3 = first_summation * y[i][0] * self.kernel_function(x[0], x[0])
            term4 = second_summation
            term5 = first_summation * self.kernel_function(x[i], x[0])
            term6 = third_summation
            
            final_barrier = -(self.epsilon / time) * ((1/alpha_values[i]) + (y[i][0]/first_summation))
            final_svm = final_barrier - (term2 - term1) + term3 - (y[i][0] * (term4 + term5)) + term6
            temp_alphas.append(final_svm[0])
        
        
        for i in range(1, length):
            summation += y[i][0] * temp_alphas[i-1]
        temp_alphas.insert(0, summation)
        
        return np.array(temp_alphas).reshape(-1, 1)
#             alphas[i] += 0.001 * differentiation_result

                
#         for i in range(1, alpha_lengths):
#             summation += alphas[i] * y[i] * y[0]
#         alphas[0] = -summation
        
     
    # ----------------------------------------------------------------------------------------------------------------------
        
    def barrier_svm(self, x, y):
        m, k = x.shape
        total_iterations = 200
        
        # Generating alphas satisfying the constraint
        while True:
            alpha_array = np.random.random_sample((m,))
            alpha1 = self.check_constraint(y, alpha_array)
            
            # If constraint is met, we change alpha_1 and break. Otherwise we continue
            if alpha1:
                alpha_array[0] = alpha1
                break
        alpha_array = alpha_array.reshape(-1, 1)
        # Got the initial feasible choice for alphas at t=0
        # Now computing the unconstrained maximizer using SVM's changes equation and barrier method.
        # self.barrier_equation(y, alpha_array)
        
        print(alpha_array)
        
        for time in range(1, total_iterations):
#             print(time)
            # Using time to decrease the power of barrier (epsilon)
            alpha_array = self.update_alphas(x, y, alpha_array, time)
        
        print(alpha_array)
        
# Testing
d_svm = dual_svm()
d_svm.barrier_svm(np.array([[1, 1], [-1, 1], [-1, -1], [1, -1]]), np.array([[-1], [1], [-1], [1]]))

[array([-5.04301988]), array([5.00319026]), array([4.96336065]), array([5.00319026])]


In [1315]:
class SVM: 
    
    def __init__(self):
        self.epsilon = 1
        self.learning_rate = 0.01
        
    def kernel_function(self, x1, x2):
        return (1 + np.dot(x1, x2)) ** 2
    
    def differentiation(self, x, y, alphas, time, j):
        length = len(alphas)
        
        first_sum, second_sum, third_sum = 0, 0, 0
        for i in range(1, length):
            first_sum += alphas[i] * y[i]
            second_sum += alphas[i] * y[i] * self.kernel_function(x[i], x[0])
            third_sum += alphas[i] * y[i] * self.kernel_function(x[i], x[j]) * y[j]
        
        barrier_term = (self.epsilon / 2**time) * ((1 / alphas[j]) + y[j] / first_sum)
        
        final_differentiation_result = - 1 + (y[j] * y[0]) + ((self.kernel_function(x[0], x[0]) * first_sum * y[j]) - y[j] * (second_sum + (self.kernel_function(x[j], x[0]) * first_sum)) + third_sum) + barrier_term
                
        return final_differentiation_result

    def barrier_svm(self, x, y, alphas, time):
        length = len(alphas)
        temp_alphas = np.zeros((length - 1, 1))
        summation, count = 0, 0
           
        for current_alpha in range(1, length):
                
            result = self.differentiation(x, y, alphas, time, current_alpha)
            if abs(result) < 0.00001:
                count += 1
            else:
                temp_alphas[current_alpha - 1] = result
                
        if count == 3:
            return alphas, False
        else:
            new_alphas = np.array(alphas[1:]).reshape(3, 1)
            new_alphas = list(new_alphas - self.learning_rate * temp_alphas)

            for i in range(1, len(alphas)):
                summation = summation + (y[i] * new_alphas[i - 1])
            new_alphas.insert(0, -summation)
            return new_alphas, True

In [1319]:
x = [[1, -1], [1, 1], [-1, 1], [-1, -1]]
y = [1, -1, 1, -1]
alphas = []
input_length = len(y)
for i in range(input_length):
    alphas.append(random.random())

svm = SVM()
for time in range(1, 500):
    alphas, flag = svm.barrier_svm(x, y, alphas, time)
    if not flag:
        break
print("Alpha 1 : {} \nAlpha 2 : {} \nAlpha 3 : {} \nAlpha 4 : {}".format(alphas[0][0], alphas[1][0], alphas[2][0], alphas[3][0]))
print("It took {} iterations to converge.".format(time))

Alpha 1 : 0.12500029218854872 
Alpha 2 : 0.12500083737602033 
Alpha 3 : 0.12500135776049745 
Alpha 4 : 0.12500081257302584
It took 154 iterations to converge.
